## Reading Data

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [2]:
mnist_train = dsets.MNIST(root="data/MNIST/", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root="data/MNIST/", train=False, transform=transforms.ToTensor(), download=True)

In [3]:
device = 'cuda' # 'cpu' or 'cuda'?
training_epochs = 15
batch_size = 100
learning_rate = 0.001
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          drop_last=True)

## Convoluational Neural Network

In [30]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__() # CNN, self 안하면 학습이 안돼요? 아뇨 멀쩡히 돼요..
        #(1,28,28)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        #(8,14,14)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        #(64,7,7)
        self.fc = nn.Linear(64 * 7 * 7, 10, bias=True)
        
        nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out # 이거 안하면 또 안돌아가..

model = CNN().to(device)

In [31]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch

    print("[Epoch: {:04d}] Cost: {:.6f}".format(epoch + 1, avg_cost))
#     print("Epoch: {:04d}/{} Cost: {:.6f} Accuracy: {:.2f}%".format(epoch + 1, training_epochs, avg_cost, accuracy.item() * 100))

[Epoch: 0001] Cost: 0.219951
[Epoch: 0002] Cost: 0.062640
[Epoch: 0003] Cost: 0.045781
[Epoch: 0004] Cost: 0.036182
[Epoch: 0005] Cost: 0.030234
[Epoch: 0006] Cost: 0.025261
[Epoch: 0007] Cost: 0.020665
[Epoch: 0008] Cost: 0.018126
[Epoch: 0009] Cost: 0.015228
[Epoch: 0010] Cost: 0.013044
[Epoch: 0011] Cost: 0.011795
[Epoch: 0012] Cost: 0.009341
[Epoch: 0013] Cost: 0.008816
[Epoch: 0014] Cost: 0.006869
[Epoch: 0015] Cost: 0.006907


In [32]:
with torch.no_grad():
    X_test = mnist_test.data.view(len(mnist_test),1,28,28).float().to(device) # .view(len(mnist_test),1,28,28)
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    
    print("Accuracy: {:.2f}%".format(accuracy.item() * 100))

Accuracy: 98.80%
